# Importing Libraries

In [1]:
import numpy as np
import os
import cv2
import random

In [2]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Data Preparation

In [3]:
def extract_label(img_path, train=True):
    filename, _ = os.path.splitext(os.path.basename(img_path))
    subject_id, etc = filename.split('__')
    
    if train:
        gender, lr, finger, alteration, _ = etc.split('_')
    else:
        gender, lr, finger, alteration = etc.split('_')
    
    '''gender = 0 if gender == 'M' else 1
    lr = 0 if lr == 'Left' else 1'''

    finger_dict = {'thumb': 0, 'index': 1, 'middle': 2, 'ring': 3, 'little': 4}
    finger = finger_dict.get(finger, -1)
    
    real = 1 if alteration == 'Real' else 0

    return np.array([finger], dtype=np.uint16), np.array([real], dtype=np.uint16)

img_size = 32

def loading_data(path, train):
    print("loading data from:", path)
    data = []
    labels_finger = []
    labels_real = []
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(img_array, (img_size, img_size))
            label_finger, label_real = extract_label(os.path.join(path, img), train)
            data.append(img_resize)
            labels_finger.append(label_finger)
            labels_real.append(label_real)
        except Exception as e:
            pass
    return np.array(data, dtype=np.float32), np.array(labels_finger), np.array(labels_real)

In [4]:
Real_path = "C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Real"
Easy_path = "C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Altered-Easy"
Medium_path = "C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Altered-Medium"
Hard_path = "C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Altered-Hard"

Easy_data, Easy_labels_finger, Easy_labels_real = loading_data(Easy_path, train=True)
Medium_data, Medium_labels_finger, Medium_labels_real = loading_data(Medium_path, train=True)
Hard_data, Hard_labels_finger, Hard_labels_real = loading_data(Hard_path, train=True)
test_data, test_labels_finger, test_labels_real = loading_data(Real_path, train=False)

# Concatenate data and labels
data = np.concatenate([Easy_data, Medium_data, Hard_data], axis=0)
labels_finger = np.concatenate([Easy_labels_finger, Medium_labels_finger, Hard_labels_finger], axis=0)

# Reshape labels to be 1D arrays
labels_finger = labels_finger.flatten()

# Convert labels to categorical
train_labels_finger = to_categorical(labels_finger, num_classes=5)

loading data from: C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Altered-Easy
loading data from: C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Altered-Medium
loading data from: C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Altered-Hard
loading data from: C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Real


In [5]:
# Normalize the data
data = data.astype('float32') / 255.0

# Manually split the data into training and validation sets
X_train, X_val, y_train_finger, y_val_finger = train_test_split(
    data, train_labels_finger, test_size=0.2, random_state=42
)

In [6]:
# Check shapes of your data and labels
print("Shapes:")
print("X_train shape:", X_train.shape)
print("y_train_finger shape:", y_train_finger.shape)

from keras.utils import to_categorical

# Assuming num_gender_classes = 2 and num_finger_classes = 5
num_finger_classes = 5

# Convert to categorical
y_train_finger_categorical = to_categorical(y_train_finger[:, 0], num_classes=num_finger_classes)

# Verify shapes after conversion
print("After conversion:")
print("y_train_finger_categorical shape:", y_train_finger_categorical.shape)

Shapes:
X_train shape: (39416, 32, 32)
y_train_finger shape: (39416, 5)
After conversion:
y_train_finger_categorical shape: (39416, 5)


# Data Generators

In [8]:
# Create ImageDataGenerator instances
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

val_datagen = ImageDataGenerator()

# Create generators
train_generator = train_datagen.flow(X_train, y_train_finger_categorical, batch_size=64)
val_generator = val_datagen.flow(X_val, y_val_finger, batch_size=64)

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (39416, 32, 32))

# Model for Gender and Finger Classification

In [12]:
# Build the model for gender and finger classification
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu',input_shape=(img_size, img_size, 1)),
    BatchNormalization(),
    MaxPooling2D(2),

    Dropout(0.4),

    Conv2D(64, 3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2),

    Flatten(),
    Dense(128, activation='relu'),    

    Dense(5, activation='softmax', name='finger_output')
])


model.summary()
model.compile(optimizer=optimizers.Adam(1e-3), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train_finger_categorical, 
                    batch_size=64, epochs=5, 
                    validation_data=[X_val, y_val_finger], 
                    callbacks=[early_stopping_cb], verbose=1)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        320       
                                                                 
 batch_normalization_3 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_4 (Bat  (None, 16, 16, 64)       

# Model for Real vs. Altered Classification

In [16]:
# Reshape labels to be 1D arrays for real vs. altered classification
alteration_labels = np.concatenate([np.ones(len(Easy_data)), np.ones(len(Medium_data)), np.ones(len(Hard_data))], axis=0)
alteration_labels = alteration_labels.flatten()

# Convert labels to categorical
train_labels_real = to_categorical(alteration_labels, num_classes=2)

# Split the data into training and validation sets
X_train_real, X_val_real, y_train_real, y_val_real = train_test_split(
    data, train_labels_real, test_size=0.3, random_state=32
)
'''
# Create generators for real vs. altered classification
train_generator_real = train_datagen.flow(X_train_real, y_train_real, batch_size=64)
val_generator_real = val_datagen.flow(X_val_real, y_val_real, batch_size=64)'''

# Build the model for real vs. altered classification
real_model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D(2),
    Conv2D(32, 3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),

    Dense(2, activation='softmax', name='real_output')
])


real_model.summary()
real_model.compile(optimizer=optimizers.Adam(1e-3), 
                   loss='categorical_crossentropy', 
                   metrics=['accuracy'])

early_stopping_cb_real = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history_real = real_model.fit(X_train_real, y_train_real, batch_size=64, epochs=5, 
                              validation_data=[X_val_real, y_val_real], 
                              callbacks=[early_stopping_cb_real], verbose=1)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        320       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 32)        9248      
                                                                 
 batch_normalization_7 (Bat  (None, 16, 16, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 8, 8, 32)          0         
 ng2D)                                                           
                                                      

# Prediciting Input Image

In [ ]:
def predict_image(model, real_model, img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_size, img_size))
    img = img.astype(np.float32) / 255.0
    img = img.reshape(1, img_size, img_size, 1)

    finger_prediction = model.predict(img)
    real_prediction = real_model.predict(img)

    finger_dict = {0: 'thumb', 1: 'index', 2: 'middle', 3: 'ring', 4: 'little'}
    finger = finger_dict[np.argmax(finger_prediction)]
    is_real = 'Real' if np.argmax(real_prediction) == 0 else 'Altered'

    return finger, is_real

In [ ]:
# Example usage of predict_image function
img_path = "C:/Users/marve/Vidhyuth/Programming/Datasets/Fingerprint_images/SOCOFing/Real/1__M_Left_little_finger.BMP"
gender, finger, is_real = predict_image(model, real_model, img_path)
print(f"Gender: {gender}, Finger: {finger}, Real/Altered: {is_real}")